<a href="https://colab.research.google.com/github/patrickhamzaokello/ColabFavourites/blob/main/Advance_Intent_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import torch
from sentence_transformers import SentenceTransformer
import spacy



In [3]:
class IntentClassifier:
    def __init__(self, intents, default_intent='menu'):
        self.intents = intents
        self.default_intent = default_intent

        # Initialize models
        self.sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')
        self.nlp = spacy.load('en_core_web_sm')
        self.tfidf = TfidfVectorizer()

        # Prepare intent embeddings
        self.prepare_intent_embeddings()

    def prepare_intent_embeddings(self):
        # Flatten intents for embedding
        self.all_examples = []
        self.intent_map = []
        for intent, examples in self.intents.items():
            self.all_examples.extend(examples)
            self.intent_map.extend([intent] * len(examples))

        # Create embeddings for all examples
        self.example_embeddings = self.sentence_transformer.encode(self.all_examples)

        # Create TF-IDF matrix
        self.tfidf_matrix = self.tfidf.fit_transform(self.all_examples)

    def classify_with_transformer(self, user_message, threshold=0.6):
        # Get message embedding
        message_embedding = self.sentence_transformer.encode([user_message])

        # Calculate similarities
        similarities = cosine_similarity(message_embedding, self.example_embeddings)[0]

        # Get best match
        best_match_idx = np.argmax(similarities)
        best_similarity = similarities[best_match_idx]

        if best_similarity >= threshold:
            return self.intent_map[best_match_idx], best_similarity
        return self.default_intent, best_similarity

    def classify_with_tfidf(self, user_message, threshold=0.3):
        # Transform user message
        message_tfidf = self.tfidf.transform([user_message])

        # Calculate similarities
        similarities = cosine_similarity(message_tfidf, self.tfidf_matrix)[0]

        # Get best match
        best_match_idx = np.argmax(similarities)
        best_similarity = similarities[best_match_idx]

        if best_similarity >= threshold:
            return self.intent_map[best_match_idx], best_similarity
        return self.default_intent, best_similarity

    def classify_with_spacy(self, user_message, threshold=0.7):
        doc = self.nlp(user_message.lower())

        best_similarity = 0
        best_intent = self.default_intent

        for intent, examples in self.intents.items():
            for example in examples:
                example_doc = self.nlp(example.lower())
                similarity = doc.similarity(example_doc)

                if similarity > best_similarity:
                    best_similarity = similarity
                    best_intent = intent

        if best_similarity >= threshold:
            return best_intent, best_similarity
        return self.default_intent, best_similarity

    def ensemble_classify(self, user_message, weights={'transformer': 0.5, 'tfidf': 0.3, 'spacy': 0.2}):
        # Get classifications from all methods
        transformer_intent, transformer_score = self.classify_with_transformer(user_message)
        tfidf_intent, tfidf_score = self.classify_with_tfidf(user_message)
        spacy_intent, spacy_score = self.classify_with_spacy(user_message)

        # Weight the scores
        weighted_scores = {
            transformer_intent: transformer_score * weights['transformer'],
            tfidf_intent: tfidf_score * weights['tfidf'],
            spacy_intent: spacy_score * weights['spacy']
        }

        # Get the intent with highest weighted score
        final_intent = max(weighted_scores.items(), key=lambda x: x[1])[0]

        return final_intent



In [10]:
intents = {
    'open_account': [
        'open account',
        'create account',
        'new account',
        'start account',
        'register account',
        'apply for account',
        'open savings account',
        'open checking account',
        'open current account',
        'create business account',
        'open joint account',
        'start fixed deposit',
        'open fixed deposit account',
        'create salary account',
        'open student account',
        'what documents needed for account',
        'account opening requirements',
        'minimum balance for account',
        'account types available',
        'business account requirements'
    ],

    'bill_payment': [
        'pay bill',
        'pay utility bill',
        'pay electricity',
        'pay water bill',
        'pay tax',
        'pay tv subscription',
        'pay dstv',
        'pay gotv',
        'pay startime',
        'pay netflix',
        'pay yaka',
        'buy electricity token',
        'purchase power token',
        'pay school fees',
        'pay university fees',
        'pay cable tv',
        'pay internet bill',
        'pay phone bill',
        'pay government tax',
        'pay property tax',
        'pay municipal fees',
        'pay insurance premium',
        'utility payment',
        'bill settlement',
        'clear pending bills'
    ],

    'merchant_services': [
        'pay merchant',
        'pay shop',
        'pay store',
        'merchant payment',
        'business payment',
        'pay vendor',
        'vendor registration',
        'register as merchant',
        'become merchant',
        'merchant account setup',
        'merchant requirements',
        'pos machine application',
        'get pos terminal',
        'payment integration',
        'online payment setup',
        'merchant fees',
        'commission rates',
        'payment gateway',
        'e-commerce integration',
        'online store integration'
    ],

    'account_inquiry': [
        'check balance',
        'account balance',
        'view statement',
        'get statement',
        'transaction history',
        'recent transactions',
        'account status',
        'account details',
        'account number',
        'branch details',
        'swift code',
        'routing number',
        'bank identifier',
        'account terms',
        'interest rates',
        'charges and fees',
        'minimum balance',
        'account limits',
        'withdrawal limits',
        'transfer limits'
    ],

    'transfer_money': [
        'send money',
        'transfer funds',
        'wire transfer',
        'international transfer',
        'local transfer',
        'send to account',
        'transfer to mobile',
        'mobile money transfer',
        'send to phone number',
        'instant transfer',
        'rtgs transfer',
        'eft transfer',
        'swift transfer',
        'cross border transfer',
        'transfer to bank',
        'send abroad',
        'remittance',
        'foreign transfer',
        'domestic transfer',
        'internal transfer'
    ],

    'complaints': [
        'file complaint',
        'raise issue',
        'report problem',
        'transaction failed',
        'payment failed',
        'wrong debit',
        'unauthorized transaction',
        'double charge',
        'failed transfer',
        'token not received',
        'missing payment',
        'wrong amount charged',
        'refund request',
        'dispute transaction',
        'card blocked',
        'account locked',
        'cannot access account',
        'app not working',
        'website down',
        'transfer stuck'
    ],

    'card_services': [
        'apply for card',
        'new card request',
        'credit card application',
        'debit card request',
        'replace card',
        'lost card',
        'stolen card',
        'block card',
        'activate card',
        'card pin reset',
        'change card pin',
        'card limits',
        'increase limit',
        'international usage',
        'card charges',
        'credit limit',
        'card delivery status',
        'virtual card',
        'prepaid card',
        'card renewal'
    ],

    'loan_services': [
        'apply for loan',
        'loan application',
        'personal loan',
        'business loan',
        'home loan',
        'car loan',
        'education loan',
        'loan requirements',
        'loan interest rate',
        'loan terms',
        'loan status',
        'loan balance',
        'repayment schedule',
        'loan calculator',
        'early repayment',
        'loan closure',
        'collateral requirements',
        'loan eligibility',
        'mortgage application',
        'refinance loan'
    ],

    'digital_banking': [
        'activate internet banking',
        'register mobile banking',
        'reset password',
        'forgot password',
        'change password',
        'app download',
        'login issues',
        'cannot login',
        'biometric setup',
        'fingerprint login',
        'face id setup',
        'device registration',
        'secure key',
        'otp not received',
        'verification code',
        'app update',
        'technical support',
        'online access',
        'digital token',
        'security settings'
    ],

    'investment_services': [
        'open investment account',
        'start fixed deposit',
        'buy shares',
        'purchase bonds',
        'invest funds',
        'investment options',
        'investment returns',
        'portfolio management',
        'wealth management',
        'mutual funds',
        'treasury bills',
        'stock trading',
        'investment advice',
        'market rates',
        'dividend payment',
        'investment calculator',
        'risk assessment',
        'investment strategy',
        'retirement planning',
        'financial planning'
    ]
}

In [12]:
# Example usage
def main():
    # Initialize classifier

    classifier = IntentClassifier(intents)

    # Example messages
    # test_messages = [
    #     "I want to open a new account",
    #     "My Yaka payment failed but money was deducted",
    #     "How do I receive money from USA?",
    #     "Need to pay my electricity bill"
    # ]

    test_messages = [
    "Hello",
    "I have made a payment to yaka, but i didn't receive a token",
    "Please i need help",
    "how can i pay taxes",
    "i need help reseting my app password",
    "What can I do here?",
    "how to send dollars to my bank account"
]

    # Test classification
    for message in test_messages:
        result = classifier.ensemble_classify(message)
        print(f"Message: {message}")
        print(f"Classified Intent: {result}\n")

if __name__ == "__main__":
    main()

<ipython-input-3-9a1c45918196>:67: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = doc.similarity(example_doc)


Message: Hello
Classified Intent: menu



<ipython-input-3-9a1c45918196>:67: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = doc.similarity(example_doc)


Message: I have made a payment to yaka, but i didn't receive a token
Classified Intent: bill_payment



<ipython-input-3-9a1c45918196>:67: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = doc.similarity(example_doc)


...

Message: i need help reseting my app password
Classified Intent: digital_banking



<ipython-input-3-9a1c45918196>:67: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = doc.similarity(example_doc)


Message: What can I do here?
Classified Intent: open_account



<ipython-input-3-9a1c45918196>:67: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = doc.similarity(example_doc)


Message: how to send dollars to my bank account
Classified Intent: transfer_money

